# BiH Political Data Collection

Voting data for the following years obtained from the [Central Election Commission BiH](https://www.izbori.ba/?Lang=6). Used to control for ethnic concentration and policy preferences.
* [2010](https://www.izbori.ba/Finalni2010/Finalni/PredsjednistvoBiH/Nivo701702.aspx)
* [2014](https://www.izbori.ba/Potvrdjeni2014/Finalni/PredsjednistvoBiH/Nivo703.aspx)
* [2018](https://www.izbori.ba/rezultati_izbora?resId=25&langId=4#/1/2/0/0/703)
* [2022](https://www.izbori.ba/Rezultati_izbora/?resId=32&langId=4#/1/1/0/0/701)

I will be focusing on the voting data associated with the presidential elections of the country. For reference, there are three presidents in BiH, one for each major ethnic group and the Bosniak and Croat ones are voted for in FBiH. As my analysis is limited to FBiH, I will only collect data on those two presidential races.

The great thing about the presidential voting data is that the website classifies each candidate to be a Bosniak or Croat so I can avoid needing to make the decision of which party or candidate represents a specific ethnic group.

I will be scraping the data from the above websites and as they do not have a robots.txt, I will assume they are okay with me doing this.

## Prerequisites

In [132]:
import json
import requests
import sqlite3

import bs4 as bs
import pandas as pd

from config import SAVE_DIR

Going to set up the schema for the database and save the db file in the processed folder. 

In [110]:
conn = sqlite3.connect(SAVE_DIR + 'political_data.db')
conn.execute('PRAGMA foreign_keys = ON')

cursor = conn.cursor()

cursor.execute("""
        CREATE TABLE IF NOT EXISTS Years (
            year_id INTEGER PRIMARY KEY,
            year    INTEGER UNIQUE
        ) STRICT
    """)

cursor.execute("""
        CREATE TABLE IF NOT EXISTS Candidates (
            candidate_id INTEGER PRIMARY KEY,
            name         TEXT UNIQUE,
            ethnicity    TEXT
        ) STRICT
    """)

cursor.execute("""
        CREATE TABLE IF NOT EXISTS Municipalities (
            municipality_id INTEGER PRIMARY KEY,
            year_id         INTEGER,
            name            TEXT,
            FOREIGN KEY (year_id) REFERENCES Years (year_id),
            UNIQUE(name, year_id)
        ) STRICT
    """)

cursor.execute("""
        CREATE TABLE IF NOT EXISTS CandidateResults (
            result_id       INTEGER PRIMARY KEY,
            municipality_id INTEGER,
            candidate_id    INTEGER,
            votes           INTEGER,
            FOREIGN KEY (municipality_id) REFERENCES Municipalities (municipality_id),
            FOREIGN KEY (candidate_id) REFERENCES Candidates (candidate_id)
            UNIQUE(municipality_id, candidate_id, votes)
        ) STRICT
    """)

conn.commit()
conn.close()

## 2022

I will be starting with this since it is very similar to the 2018 website and is more modern than the 2014 or 2010 website which will likely require a different scraping script.

### 1: Populating Candidates with their ethnicities

Since this data is all on one page and to avoid making my scraping script more complicated I will do this by hand.

In [111]:
def add_candidates(candidates: list[tuple()]):   
    conn = sqlite3.connect(SAVE_DIR + 'political_data.db')
    conn.execute('PRAGMA foreign_keys = ON')
    
    cursor = conn.cursor()
    
    sql_query = """
            INSERT INTO Candidates 
                (name, ethnicity)
            VALUES (?, ?)
            ON CONFLICT(name) DO NOTHING
        """
    
    try:
        cursor.executemany(sql_query, candidates)
        conn.commit()
        print(f"Successfully inserted {cursor.rowcount} candidates.")
    
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    
    finally:
        conn.close()

In [112]:
candidates = [
    ("DENIS BEĆIROVIĆ - UJEDINJENI ZA SLOBODNU BOSNU I HERCEGOVINU", "B"),
    ("BAKIR IZETBEGOVIĆ - SDA - STRANKA DEMOKRATSKE AKCIJE", "B"),
    ("MIRSAD HADŽIKADIĆ - PLATFORMA ZA PROGRES", "B"),
    ("ŽELJKO KOMŠIĆ - DEMOKRATSKA FRONTA - DF", "C"),
    ("BORJANA KRIŠTO - HDZ BIH-HRVATSKA DEMOKRATSKA ZAJEDNICA BOSNE I HERCEGOVINE", "C"),
]
add_candidates(candidates)

Successfully inserted 5 candidates.


### 2: Create Scraping Script

I got the following html from inspecting the dropdown for municipalities.

In [113]:
scroll_down_html = """
<select class="ddlMenu ng-pristine ng-valid" ng-model="electoralUnitSelectedId" ng-change="race_1_ElectoralUnitChange()">
                        <option value="0">-</option>
                        <!-- ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="1" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VELIKA KLADUŠA (001)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="true" ng-repeat="eu in electoralUnitList" value="2" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope" selected="selected">CAZIN (002)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="3" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BIHAĆ (003)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="4" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BOSANSKA KRUPA (004)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="5" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BUŽIM (005)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="17" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ODŽAK (017)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="20" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOMALJEVAC - ŠAMAC (020)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="22" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ORAŠJE (022)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="25" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRADAČAC (025)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="27" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BRČKO DISTRIKT BIH (OPCIJA FBIH) (027)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="30" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BOSANSKI PETROVAC (030)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="32" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">SANSKI MOST (032)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="36" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOBOJ ISTOK (036)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="37" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOBOJ JUG (037)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="39" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TEŠANJ (039)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="42" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">MAGLAJ (042)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="44" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRAČANICA (044)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="47" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">LUKAVAC (047)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="49" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">SREBRENIK (049)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="50" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TUZLA (050)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="52" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ČELIĆ (052)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="55" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TEOČAK (055)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="57" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DRVAR (057)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="59" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KLJUČ (059)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="65" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">JAJCE (065)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="67" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOBRETIĆI (067)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="75" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ŽEPČE (075)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="77" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ZAVIDOVIĆI (077)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="78" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BANOVIĆI (078)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="79" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ŽIVINICE (079)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="80" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KALESIJA (080)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="82" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">SAPNA (082)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="84" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BOSANSKO GRAHOVO (084)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="85" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GLAMOČ (085)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="89" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DONJI VAKUF (089)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="91" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TRAVNIK (091)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="93" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ZENICA (093)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="94" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KAKANJ (094)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="95" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VAREŠ (095)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="96" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">OLOVO (096)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="98" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KLADANJ (098)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="106" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">LIVNO (106)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="107" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KUPRES (107)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="109" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BUGOJNO (109)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="110" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GORNJI VAKUF - USKOPLJE (110)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="111" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NOVI TRAVNIK (111)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="112" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VITEZ (112)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="113" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BUSOVAČA (113)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="114" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">FOJNICA (114)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="115" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KISELJAK (115)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="116" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VISOKO (116)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="117" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BREZA (117)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="118" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ILIJAŠ (118)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="124" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TOMISLAVGRAD (124)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="125" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">PROZOR - RAMA (125)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="126" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">JABLANICA (126)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="127" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KONJIC (127)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="129" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KREŠEVO (129)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="130" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">HADŽIĆI (130)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="131" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ILIDŽA (131)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="133" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NOVI GRAD SARAJEVO (133)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="135" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VOGOŠĆA (135)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="136" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">CENTAR SARAJEVO (136)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="137" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">STARI GRAD SARAJEVO (137)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="139" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NOVO SARAJEVO (139)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="141" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TRNOVO (FBIH) (141)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="143" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">PALE (FBIH) (143)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="148" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">POSUŠJE (148)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="149" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRUDE (149)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="150" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ŠIROKI BRIJEG (150)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="165" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">FOČA (FBIH) (165)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="167" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GORAŽDE (167)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="171" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">LJUBUŠKI (171)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="172" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ČITLUK (172)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="173" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ČAPLJINA (173)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="174" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NEUM (174)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="176" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">STOLAC (176)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="181" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">RAVNO (181)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="183" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">USORA (183)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="199" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRAD MOSTAR (199)</option><!-- end ngRepeat: eu in electoralUnitList -->
                    </select>
"""
soup = bs.BeautifulSoup(scroll_down_html)

To now iterate and organize them.

In [114]:
soup = bs.BeautifulSoup(scroll_down_html)
municipality_to_code = {
    option.text.split("(")[0].strip().lower() : int(option.get("value"))
    for option in soup.find_all("option")
    if int(option.get("value")) != 0
}

To make inserting into the db faster I will prefill it with municipality and year.

In [115]:
def add_year(year: int):
    conn = sqlite3.connect(SAVE_DIR + 'political_data.db')
    conn.execute('PRAGMA foreign_keys = ON')
    
    cursor = conn.cursor()
    
    sql_query = """
            INSERT INTO Years 
                (year)
            VALUES (?)
            ON CONFLICT(year) DO NOTHING
        """
    
    try:
        cursor.execute(sql_query, (year,))
        conn.commit()
        print(f"Successfully inserted {cursor.rowcount} year.")
    
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    
    finally:
        conn.close()

In [116]:
add_year(2022)

Successfully inserted 1 year.


To add the municipalities and relate them to the correct year.

In [117]:
def get_year_id(year: int):
    year_id = None
    conn = sqlite3.connect(SAVE_DIR + 'political_data.db')
    conn.execute('PRAGMA foreign_keys = ON')
    
    cursor = conn.cursor()
    
    sql_query = """
            SELECT * FROM Years
            WHERE Years.year = (?)
        """
    
    try:
        cursor.execute(sql_query, (year,))
        year_id = cursor.fetchone()
    
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    
    finally:
        conn.close()

    return year_id[0]

In [118]:
current_year_id = get_year_id(2022)
current_year_id

1

So its easier to merge later, I will encode it following the same as the other tables.

In [119]:
municipalities = list(municipality_to_code.keys())
municipalities[4], municipalities[4].encode(encoding="iso-8859-1", errors="replace").decode("UTF-8")

('bužim', 'bu?im')

In [120]:
municipalities = map(
    lambda x: x.encode(encoding="iso-8859-1", errors="replace").decode("UTF-8"), 
    municipalities
)

In [121]:
municipalities = [(name, current_year_id) for name in municipalities]
# municipalities

Now to insert the municipalites to the year.

In [124]:
conn = sqlite3.connect(SAVE_DIR + 'political_data.db')
conn.execute('PRAGMA foreign_keys = ON')

cursor = conn.cursor()

sql_query = """
        INSERT INTO Municipalities 
            (name, year_id)
        VALUES (?, ?)
        ON CONFLICT(name, year_id) DO NOTHING
    """

try:
    cursor.executemany(sql_query, municipalities)
    conn.commit()
    print(f"Successfully inserted {cursor.rowcount} municipalites.")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")
    conn.rollback()

finally:
    conn.close()

Successfully inserted 0 municipalites.


To get the municpal db ids for this year to have easier time inserting them.

In [125]:
def get_municipality_id_per_year(year: int) -> dict:
    year_id = get_year_id(year)
    municipalites_this_year = None

    conn = sqlite3.connect(SAVE_DIR + 'political_data.db')
    conn.execute('PRAGMA foreign_keys = ON')
    
    cursor = conn.cursor()
    
    sql_query = """
            SELECT * FROM Municipalities
            WHERE Municipalities.year_id = (?)
        """
    
    try:
        cursor.execute(sql_query, (year_id,))
        conn.commit()
        municipalites_this_year = cursor.fetchall()
    
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    
    finally:
        conn.close()

    return {name : municip_id for municip_id, _, name in municipalites_this_year}

In [126]:
# get_municipality_id_per_year(2022)

Need a way to get candidates and their ids to insert into CandidateResults.

In [127]:
def get_candidate_id() -> dict:
    candidates_id = None

    conn = sqlite3.connect(SAVE_DIR + 'political_data.db')
    conn.execute('PRAGMA foreign_keys = ON')
    
    cursor = conn.cursor()
    
    sql_query = """
            SELECT * FROM Candidates
        """
    
    try:
        cursor.execute(sql_query)
        conn.commit()
        candidates_id = cursor.fetchall()
    
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    
    finally:
        conn.close()

    return {name : can_id for can_id, name, _ in candidates_id}

In [128]:
# get_candidate_id()

Now to have function to simplify inserting into CandidateResults.

In [129]:
def insert_candidate_results(candidate_results, municipality, year):
    candidate_ids = get_candidate_id()
    municipality_ids = get_municipality_id_per_year(2022)    
    batch = [(municipality_ids[municipality], candidate_ids[name], votes) for name, votes in candidate_results.items()]
    
    conn = sqlite3.connect(SAVE_DIR + 'political_data.db')
    conn.execute('PRAGMA foreign_keys = ON')
    
    cursor = conn.cursor()
    
    sql_query = """
            INSERT INTO CandidateResults 
                (municipality_id, candidate_id, votes)
            VALUES (?, ?, ?)
            ON CONFLICT(municipality_id, candidate_id, votes) DO NOTHING
        """
    
    try:
        cursor.executemany(sql_query, batch)
        conn.commit()
        print(f"Successfully inserted {cursor.rowcount} candidate results for {municipality}.")
    
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()
    
    finally:
        conn.close()

### 3: Scrape Data

In [130]:
def scrape_2022():
    year = 2022
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Referer': 'https://www.izbori.ba/Rezultati_izbora/?resId=32&langId=4',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.6 Safari/605.1.15',
        'Sec-Fetch-Mode': 'cors'
    }
    scroll_down_html = """
    <select class="ddlMenu ng-pristine ng-valid" ng-model="electoralUnitSelectedId" ng-change="race_1_ElectoralUnitChange()">
                            <option value="0">-</option>
                            <!-- ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="1" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VELIKA KLADUŠA (001)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="true" ng-repeat="eu in electoralUnitList" value="2" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope" selected="selected">CAZIN (002)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="3" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BIHAĆ (003)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="4" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BOSANSKA KRUPA (004)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="5" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BUŽIM (005)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="17" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ODŽAK (017)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="20" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOMALJEVAC - ŠAMAC (020)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="22" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ORAŠJE (022)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="25" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRADAČAC (025)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="27" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BRČKO DISTRIKT BIH (OPCIJA FBIH) (027)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="30" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BOSANSKI PETROVAC (030)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="32" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">SANSKI MOST (032)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="36" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOBOJ ISTOK (036)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="37" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOBOJ JUG (037)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="39" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TEŠANJ (039)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="42" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">MAGLAJ (042)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="44" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRAČANICA (044)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="47" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">LUKAVAC (047)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="49" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">SREBRENIK (049)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="50" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TUZLA (050)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="52" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ČELIĆ (052)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="55" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TEOČAK (055)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="57" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DRVAR (057)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="59" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KLJUČ (059)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="65" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">JAJCE (065)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="67" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOBRETIĆI (067)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="75" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ŽEPČE (075)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="77" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ZAVIDOVIĆI (077)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="78" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BANOVIĆI (078)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="79" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ŽIVINICE (079)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="80" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KALESIJA (080)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="82" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">SAPNA (082)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="84" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BOSANSKO GRAHOVO (084)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="85" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GLAMOČ (085)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="89" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DONJI VAKUF (089)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="91" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TRAVNIK (091)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="93" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ZENICA (093)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="94" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KAKANJ (094)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="95" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VAREŠ (095)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="96" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">OLOVO (096)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="98" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KLADANJ (098)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="106" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">LIVNO (106)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="107" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KUPRES (107)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="109" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BUGOJNO (109)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="110" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GORNJI VAKUF - USKOPLJE (110)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="111" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NOVI TRAVNIK (111)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="112" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VITEZ (112)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="113" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BUSOVAČA (113)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="114" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">FOJNICA (114)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="115" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KISELJAK (115)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="116" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VISOKO (116)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="117" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BREZA (117)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="118" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ILIJAŠ (118)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="124" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TOMISLAVGRAD (124)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="125" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">PROZOR - RAMA (125)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="126" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">JABLANICA (126)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="127" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KONJIC (127)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="129" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KREŠEVO (129)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="130" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">HADŽIĆI (130)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="131" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ILIDŽA (131)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="133" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NOVI GRAD SARAJEVO (133)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="135" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VOGOŠĆA (135)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="136" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">CENTAR SARAJEVO (136)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="137" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">STARI GRAD SARAJEVO (137)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="139" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NOVO SARAJEVO (139)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="141" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TRNOVO (FBIH) (141)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="143" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">PALE (FBIH) (143)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="148" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">POSUŠJE (148)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="149" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRUDE (149)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="150" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ŠIROKI BRIJEG (150)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="165" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">FOČA (FBIH) (165)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="167" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GORAŽDE (167)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="171" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">LJUBUŠKI (171)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="172" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ČITLUK (172)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="173" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ČAPLJINA (173)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="174" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NEUM (174)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="176" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">STOLAC (176)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="181" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">RAVNO (181)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="183" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">USORA (183)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="199" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRAD MOSTAR (199)</option><!-- end ngRepeat: eu in electoralUnitList -->
                        </select>
    """
    soup = bs.BeautifulSoup(scroll_down_html)
    municipality_to_code = {
        option.text.split("(")[0].strip().lower() : int(option.get("value"))
        for option in soup.find_all("option")
        if int(option.get("value")) != 0
    }
    
    for municipality, code in municipality_to_code.items():
        api_url = f'https://www.izbori.ba/api_2018/race1_electoralunitcandidatesresult/"WebResult_2022GENT1_2022_4_20_14_10_43"/{code}/4'
        municipality =  municipality.encode(encoding="iso-8859-1", errors="replace").decode("UTF-8")
        
        try:
            response = requests.get(api_url, headers=headers)
            response.raise_for_status()
        
            data = response.json()
            candidate_resuslts = {result["name"] : result["totalVotes"] for result in data}
            insert_candidate_results(candidate_resuslts, municipality, year)
        
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
        except requests.exceptions.JSONDecodeError:
            print("Failed to decode JSON. The response was not JSON.")
            print(response.text)

In [131]:
scrape_2022()

Successfully inserted 5 candidate results for velika kladu?a.
Successfully inserted 5 candidate results for cazin.
Successfully inserted 5 candidate results for biha?.
Successfully inserted 5 candidate results for bosanska krupa.
Successfully inserted 5 candidate results for bu?im.
Successfully inserted 5 candidate results for od?ak.
Successfully inserted 5 candidate results for domaljevac - ?amac.
Successfully inserted 5 candidate results for ora?je.
Successfully inserted 5 candidate results for grada?ac.
Successfully inserted 5 candidate results for br?ko distrikt bih.
Successfully inserted 5 candidate results for bosanski petrovac.
Successfully inserted 5 candidate results for sanski most.
Successfully inserted 5 candidate results for doboj istok.
Successfully inserted 5 candidate results for doboj jug.
Successfully inserted 5 candidate results for te?anj.
Successfully inserted 5 candidate results for maglaj.
Successfully inserted 5 candidate results for gra?anica.
Successfully inse

### 4: Sanity check that data was inserted properly and can be loaded by pandas.

In [146]:
try:
    conn = sqlite3.connect(SAVE_DIR + "political_data.db")

    sql_query = """
        SELECT
            m.name AS municipality_name,
            y.year,
            c.name,
            c.ethnicity,
            cr.votes
        FROM 
            Municipalities as m
        JOIN
            Years AS y ON m.year_id = y.year_id
        JOIN
            CandidateResults AS cr ON m.municipality_id = cr.municipality_id
        JOIN
            Candidates AS c ON cr.candidate_id = c.candidate_id
    """

    df = pd.read_sql_query(sql_query, conn)
except sqlite3.Error as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()

df

,municipality_name,year,name,ethnicity,votes
0,velika kladu?a,2022,DENIS BEĆIROVIĆ - UJEDINJENI ZA SLOBODNU BOSNU...,B,4662
1,velika kladu?a,2022,BAKIR IZETBEGOVIĆ - SDA - STRANKA DEMOKRATSKE ...,B,3039
2,velika kladu?a,2022,ŽELJKO KOMŠIĆ - DEMOKRATSKA FRONTA - DF,C,2999
3,velika kladu?a,2022,MIRSAD HADŽIKADIĆ - PLATFORMA ZA PROGRES,B,690
4,velika kladu?a,2022,BORJANA KRIŠTO - HDZ BIH-HRVATSKA DEMOKRATSKA ...,C,190
...,...,...,...,...,...
395,grad mostar,2022,BORJANA KRIŠTO - HDZ BIH-HRVATSKA DEMOKRATSKA ...,C,21651
396,grad mostar,2022,ŽELJKO KOMŠIĆ - DEMOKRATSKA FRONTA - DF,C,9332
397,grad mostar,2022,DENIS BEĆIROVIĆ - UJEDINJENI ZA SLOBODNU BOSNU...,B,8822
398,grad mostar,2022,BAKIR IZETBEGOVIĆ - SDA - STRANKA DEMOKRATSKE ...,B,6157


The above checks out!

## 2018

### 1: Insert year into Years

In [147]:
add_year(2018)

Successfully inserted 1 year.


### 2: Insert candidates

In [148]:
candidates = [
    ("DŽAFEROVIĆ ŠEFIK - SDA - STRANKA DEMOKRATSKE AKCIJE", "B"),
    ("BEĆIROVIĆ DENIS - SDP - SOCIJALDEMOKRATSKA PARTIJA BOSNE I HERCEGOVINE", "B"),
    ("RADONČIĆ FAHRUDIN - SBB-FAHRUDIN RADONČIĆ", "B"),
    ("HADŽIKADIĆ MIRSAD - MIRSAD HADŽIKADIĆ-PLATFORMA ZA PROGRES", "B"),
    ("ŠEPIĆ SENAD - NEZAVISNI BLOK", "B"),
    ("JERLAGIĆ AMER - STRANKA ZA BOSNU I HERCEGOVINU", "B"),
    ("KOMŠIĆ ŽELJKO - DEMOKRATSKA FRONTA", "C"),
    ("ČOVIĆ DRAGAN - HDZ BIH-HRVATSKA DEMOKRATSKA ZAJEDNICA BOSNE I HERCEGOVINE", "C"),
    ("ZELENIKA DIANA - HRVATSKA DEMOKRATSKA ZAJEDNICA 1990 - HDZ 1990", "C"),
    ("FALATAR BORIŠA - NAŠA STRANKA", "C"),
    ("IVANKOVIĆ-LIJANOVIĆ JERKO - NARODNA STRANKA RADOM ZA BOLJITAK", "C"),
]
add_candidates(candidates)

Successfully inserted 11 candidates.


### 3: Reuse script from 2022 with modified api

In [149]:
def scrape_2018():
    year = 2018
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Referer': 'https://www.izbori.ba/Rezultati_izbora/?resId=32&langId=4',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.6 Safari/605.1.15',
        'Sec-Fetch-Mode': 'cors'
    }
    scroll_down_html = """
    <select class="ddlMenu ng-pristine ng-valid" ng-model="electoralUnitSelectedId" ng-change="race_1_ElectoralUnitChange()">
                        <option value="0">-</option>
                        <!-- ngRepeat: eu in electoralUnitList --><option ng-selected="true" ng-repeat="eu in electoralUnitList" value="1" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope" selected="selected">VELIKA KLADUŠA (001)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="2" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">CAZIN (002)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="3" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BIHAĆ (003)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="4" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BOSANSKA KRUPA (004)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="5" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BUŽIM (005)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="17" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ODŽAK (017)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="20" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOMALJEVAC - ŠAMAC (020)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="22" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ORAŠJE (022)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="25" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRADAČAC (025)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="27" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BRČKO DISTRIKT BIH (OPCIJA FBIH) (027)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="30" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BOSANSKI PETROVAC (030)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="32" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">SANSKI MOST (032)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="36" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOBOJ - ISTOK (036)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="37" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOBOJ - JUG (037)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="39" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TEŠANJ (039)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="42" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">MAGLAJ (042)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="44" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRAČANICA (044)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="47" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">LUKAVAC (047)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="49" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">SREBRENIK (049)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="50" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TUZLA (050)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="52" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ČELIĆ (052)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="55" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TEOČAK (055)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="57" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DRVAR (057)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="59" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KLJUČ (059)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="65" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">JAJCE (065)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="67" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DOBRETIĆI (067)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="75" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ŽEPČE (075)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="77" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ZAVIDOVIĆI (077)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="78" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BANOVIĆI (078)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="79" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ŽIVINICE (079)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="80" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KALESIJA (080)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="82" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">SAPNA (082)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="84" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BOSANSKO GRAHOVO (084)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="85" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GLAMOČ (085)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="89" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">DONJI VAKUF (089)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="91" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TRAVNIK (091)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="93" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ZENICA (093)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="94" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KAKANJ (094)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="95" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VAREŠ (095)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="96" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">OLOVO (096)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="98" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KLADANJ (098)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="106" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">LIVNO (106)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="107" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KUPRES (107)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="109" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BUGOJNO (109)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="110" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GORNJI VAKUF - USKOPLJE (110)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="111" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NOVI TRAVNIK (111)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="112" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VITEZ (112)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="113" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BUSOVAČA (113)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="114" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">FOJNICA (114)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="115" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KISELJAK (115)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="116" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VISOKO (116)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="117" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">BREZA (117)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="118" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ILIJAŠ (118)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="124" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TOMISLAVGRAD (124)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="125" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">PROZOR - RAMA (125)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="126" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">JABLANICA (126)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="127" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KONJIC (127)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="129" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">KREŠEVO (129)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="130" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">HADŽIĆI (130)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="131" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ILIDŽA (131)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="133" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NOVI GRAD SARAJEVO (133)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="135" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">VOGOŠĆA (135)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="136" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">CENTAR SARAJEVO (136)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="137" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">STARI GRAD SARAJEVO (137)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="139" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NOVO SARAJEVO (139)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="141" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">TRNOVO (FBIH) (141)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="143" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">PALE (FBIH) (143)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="148" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">POSUŠJE (148)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="149" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRUDE (149)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="150" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ŠIROKI BRIJEG (150)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="165" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">FOČA (FBIH) (165)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="167" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GORAŽDE (167)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="171" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">LJUBUŠKI (171)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="172" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ČITLUK (172)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="173" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">ČAPLJINA (173)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="174" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">NEUM (174)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="176" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">STOLAC (176)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="181" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">RAVNO (181)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="183" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">USORA (183)</option><!-- end ngRepeat: eu in electoralUnitList --><option ng-selected="false" ng-repeat="eu in electoralUnitList" value="199" ng-class="{inactiveEu: !eu.active}" class="ng-binding ng-scope">GRAD MOSTAR (199)</option><!-- end ngRepeat: eu in electoralUnitList -->
                    </select>
    """
    soup = bs.BeautifulSoup(scroll_down_html)
    municipality_to_code = {
        option.text.split("(")[0].strip().lower() : int(option.get("value"))
        for option in soup.find_all("option")
        if int(option.get("value")) != 0
    }
    
    for municipality, code in municipality_to_code.items():
        api_url = f' https://www.izbori.ba/api_2018/race1_electoralunitcandidatesresult/"WebResult_2018GEN_2018_10_4_15_40_5"/{code}/4'
        municipality =  municipality.encode(encoding="iso-8859-1", errors="replace").decode("UTF-8")
        
        try:
            response = requests.get(api_url, headers=headers)
            response.raise_for_status()
        
            data = response.json()
            candidate_resuslts = {result["name"] : result["totalVotes"] for result in data}
            insert_candidate_results(candidate_resuslts, municipality, year)
        
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
        except requests.exceptions.JSONDecodeError:
            print("Failed to decode JSON. The response was not JSON.")
            print(response.text)

In [ ]:
# TODO take the names without hyphens from 2022 as keys and remove any "-" 
    # from new data and non political data so its easier to merge.
    # Still might have to do some merge resolution tho,
    # especially on name mismatches with ? chars :(
# Have text preprocessing all done in one place and reuse this throughout

In [150]:
scrape_2018()

Successfully inserted 11 candidate results for velika kladu?a.
Successfully inserted 11 candidate results for cazin.
Successfully inserted 11 candidate results for biha?.
Successfully inserted 11 candidate results for bosanska krupa.
Successfully inserted 11 candidate results for bu?im.
Successfully inserted 11 candidate results for od?ak.
Successfully inserted 11 candidate results for domaljevac - ?amac.
Successfully inserted 11 candidate results for ora?je.
Successfully inserted 11 candidate results for grada?ac.
Successfully inserted 11 candidate results for br?ko distrikt bih.
Successfully inserted 11 candidate results for bosanski petrovac.
Successfully inserted 11 candidate results for sanski most.


KeyError: 'doboj - istok'

### 4: Sanity check

In [158]:
try:
    conn = sqlite3.connect(SAVE_DIR + "political_data.db")

    sql_query = """
        SELECT
            m.name AS municipality_name,
            y.year,
            c.name,
            c.ethnicity,
            cr.votes
        FROM 
            Municipalities as m
        JOIN
            Years AS y ON m.year_id = y.year_id
        JOIN
            CandidateResults AS cr ON m.municipality_id = cr.municipality_id
        JOIN
            Candidates AS c ON cr.candidate_id = c.candidate_id
    """

    df = pd.read_sql_query(sql_query, conn)
except sqlite3.Error as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()

df

,municipality_name,year,name,ethnicity,votes
0,velika kladu?a,2022,DENIS BEĆIROVIĆ - UJEDINJENI ZA SLOBODNU BOSNU...,B,4662
1,velika kladu?a,2022,BAKIR IZETBEGOVIĆ - SDA - STRANKA DEMOKRATSKE ...,B,3039
2,velika kladu?a,2022,ŽELJKO KOMŠIĆ - DEMOKRATSKA FRONTA - DF,C,2999
3,velika kladu?a,2022,MIRSAD HADŽIKADIĆ - PLATFORMA ZA PROGRES,B,690
4,velika kladu?a,2022,BORJANA KRIŠTO - HDZ BIH-HRVATSKA DEMOKRATSKA ...,C,190
...,...,...,...,...,...
527,sanski most,2022,FALATAR BORIŠA - NAŠA STRANKA,C,102
528,sanski most,2022,ČOVIĆ DRAGAN - HDZ BIH-HRVATSKA DEMOKRATSKA ZA...,C,64
529,sanski most,2022,JERLAGIĆ AMER - STRANKA ZA BOSNU I HERCEGOVINU,B,56
530,sanski most,2022,IVANKOVIĆ-LIJANOVIĆ JERKO - NARODNA STRANKA RA...,C,49


In [160]:
df[df['municipality_name'].str.contains("doboj", case=False)]

,municipality_name,year,name,ethnicity,votes
60,doboj istok,2022,DENIS BEĆIROVIĆ - UJEDINJENI ZA SLOBODNU BOSNU...,B,2018
61,doboj istok,2022,BAKIR IZETBEGOVIĆ - SDA - STRANKA DEMOKRATSKE ...,B,2017
62,doboj istok,2022,ŽELJKO KOMŠIĆ - DEMOKRATSKA FRONTA - DF,C,1096
63,doboj istok,2022,MIRSAD HADŽIKADIĆ - PLATFORMA ZA PROGRES,B,132
64,doboj istok,2022,BORJANA KRIŠTO - HDZ BIH-HRVATSKA DEMOKRATSKA ...,C,11
65,doboj jug,2022,DENIS BEĆIROVIĆ - UJEDINJENI ZA SLOBODNU BOSNU...,B,707
66,doboj jug,2022,ŽELJKO KOMŠIĆ - DEMOKRATSKA FRONTA - DF,C,697
67,doboj jug,2022,BAKIR IZETBEGOVIĆ - SDA - STRANKA DEMOKRATSKE ...,B,404
68,doboj jug,2022,MIRSAD HADŽIKADIĆ - PLATFORMA ZA PROGRES,B,60
69,doboj jug,2022,BORJANA KRIŠTO - HDZ BIH-HRVATSKA DEMOKRATSKA ...,C,6


## 2014

## 2010